In [2]:
import pandas as pd
import re

list_data_csv = pd.read_csv('input/input.csv')
list_data_csv


,address
0,東京都大田区大森本町1-8-12-S-1203
1,東京都大田区山王1-7-8山王薬局1丁目店
2,東京都台東区浅草橋1-32-6-2F
3,東京都品川区高輪4-8-10-5階
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号
...,...
57,名古屋市熱田区神宮四丁目７番２７号
58,名古屋市熱田区神宮四丁目７番２７号Bビル
59,名古屋市港区空見町36番地
60,名古屋市港区十一屋3丁目63番地


In [2]:
type(list_data_csv['address'][0])

str

In [3]:
def find_prefacture(string: str):
    # 北海道,大阪府,京都府,東京都は別扱い
    special_prefactures: list = ["北海道","大阪府","京都府","東京都"]
    for _ in special_prefactures:
        if _ in string:
            return (_,)

def erase_special_address_expression(string: str, index: int):
    size: int = len(string)
    # 丁目を置換することのみ行う
    if index == 1:
        res: str = ""
        i :int = 0
        while i < (size):
            if i == size-1:
                res = res + string[i]
                i += 1
            else:
                if(string[i]=='丁' and string[i+1]=='目'):
                    # 丁目店などは特殊ケースとして除外
                    if i+2 < size and string[i+2] == "店":
                        res = res + string[i]
                        i += 1
                        continue
                    res = res + ("-")
                    i+=2
                else: 
                    res = res + (string[i])
                    i += 1
        return res
    # 番地を置換することのみ行う
    if index == 2:
        res: str = ""
        i :int = 0
        while i < size:
            if i == size-1:
                res = res + string[i]
                i += 1
            else:
                if(string[i]=="番" and string[i+1]=="地"):
                    res = res + ("-")
                    i+=2
                else: 
                    res = res + (string[i])
                    i+=1
        return res
    # 番を置換することのみ行う
    if index == 3:
        res: str = ""
        for i in range(size):
                if(string[i]=="番"): res = res + ("-")
                else: res = res + (string[i])
        return res
    # の を置換する
    if index == 4:
        res: str = ""

        numbers: list = []
        for i in range(10):
            numbers.append(str(i))
        
        for i in range(size):
            if i > 0 and i+1 < size and string[i]=="の" and string[i-1] in numbers:
                res = res + "-"
            else: res = res + string[i]
        return res

    
def  replace_japanese_address_expression(string: str):
    # ~丁目,~番,~番地を-に変換
    # 注意: 番地 -> 番 の順番に行わないとデータが壊れる
    if "丁目" in string:
        string = erase_special_address_expression(string, 1)
    if "番地" in string:
        string = erase_special_address_expression(string, 2)
    if "番" in string:
        string = erase_special_address_expression(string, 3)
    if "の" in string:
        string = erase_special_address_expression(string, 4)
    return string

In [4]:
replaced_address_data: list = []

def erase_last_hyphen(string: str):
    if string[-1]=="-":
        return string[0:len(string)-1]
    else: return string

def replace_slash_with_hyphen(string: str):
    # 空白を取り除く
    string = string.replace(' ', '')
    string = string.replace('　', '')
    # 正規表現で/を-に置き換える
    return re.sub("/", "-", string)

def japanese_style_number_to_number(string: str):
    def function(char: str):
        dictionary_japanese_style_number: dict = {"一":"1","二":"2","三":"3","四":"4","五":"5","六":"6","七":"7","八":"8","九":"9","十":"10","〇":"0"}
        return dictionary_japanese_style_number[char]
    def full_width_to_half_width(char :str):
        mapping_dictionary: dict = {"１":"1","２":"2","３":"3","４":"4","５":"5","６":"6","７":"7","８":"8","９":"9","０":"10"}
        full_width_number: list = ["１","２","３","４","５","６","７","８","９","０"]
        if char in full_width_number:
            return mapping_dictionary[c]
        else: return char
    japanese_style_number: list = ["一","二","三","四","五","六","七","八","九","十","〇"]
    
    res: str = ""
    for c in string:
        if c in japanese_style_number:
            res = res + full_width_to_half_width(function(c))
        else: res = res + full_width_to_half_width(c)
    return res

def full_width_hyphen_to_half_width_hyphen(string: str):
    res: str = ""
    for char in string:
        if char == "ー":
            res = res + "-"
        else: res = res + char
    return res

def operation(string:str):
    return full_width_hyphen_to_half_width_hyphen(japanese_style_number_to_number(replace_slash_with_hyphen(erase_last_hyphen(replace_japanese_address_expression(string)))))

for address_data in list_data_csv['address']:
    replaced_address_data.append(operation(address_data))

pd.DataFrame(replaced_address_data)



,0
0,東京都大田区大森本町1-8-12-S-1203
1,東京都大田区山王1-7-8山王薬局1丁目店
2,東京都台東区浅草橋1-32-6-2F
3,東京都品川区高輪4-8-10-5階
4,東京都大田区西6郷4-30-7音金6郷マンション1-C号
...,...
57,名古屋市熱田区神宮4-7-27号
58,名古屋市熱田区神宮4-7-27号Bビル
59,名古屋市港区空見町36
60,名古屋市港区101屋3-63


In [5]:
data: dict = {}
data['original'] = list_data_csv['address']

def extract_prefacture(string: str):
    prefactures: list = ["北海道","青森県","岩手県","宮城県","秋田県","山形県","福島県","茨城県","栃木県","群馬県","埼玉県","千葉県","東京都","神奈川県","新潟県","富山県","石川県","福井県","山梨県","長野県","岐阜県","静岡県","愛知県","三重県","滋賀県","京都府","大阪府","兵庫県","奈良県","和歌山県","鳥取県","島根県","岡山県","広島県","山口県","徳島県","香川県","愛媛県","高知県","福岡県","佐賀県","長崎県","熊本県","大分県","宮崎県","鹿児島県","沖縄県"]
    
    for prefacture in prefactures:
        if prefacture in string:
            return (prefacture,string.replace(prefacture, ""))
    # 県名が存在しない
    return ("", string)

prefactures: list = []
non_prefacture_address_data: list = []
for string in replaced_address_data:
    tuple_data: tuple = extract_prefacture(string)
    prefactures.append(tuple_data[0])
    non_prefacture_address_data.append(tuple_data[1])

data["prefacture"] = prefactures
# data["other"] = non_prefacture_address_data

pd.DataFrame(data)

,original,prefacture
0,東京都大田区大森本町1-8-12-S-1203,東京都
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都
2,東京都台東区浅草橋1-32-6-2F,東京都
3,東京都品川区高輪4-8-10-5階,東京都
4,東京都大田区西6郷4-30-7音金6郷マンション1-C号,東京都
...,...,...
57,名古屋市熱田区神宮4-7-27号,
58,名古屋市熱田区神宮4-7-27号Bビル,
59,名古屋市港区空見町36,
60,名古屋市港区101屋3-63,


In [6]:
non_prefacture_address_data
# ここから市、区を取り除く

cities: list = []
non_city_address_data: list = []

for city_address_data in non_prefacture_address_data:
    if "市" in city_address_data:
        index: int = city_address_data.find("市")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    elif "区" in city_address_data:
        index: int = city_address_data.find("区")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    elif "郡" in city_address_data:
        index: int = city_address_data.find("郡")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    elif "町" in city_address_data:
        index: int = city_address_data.find("町")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    elif "村" in city_address_data:
        index: int = city_address_data.find("村")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    else:
        cities.append("")
        non_city_address_data.append(city_address_data)

data['city'] = cities
# data['other'] = non_city_address_data


pd.DataFrame(data)

,original,prefacture,city
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区
3,東京都品川区高輪4-8-10-5階,東京都,品川区
4,東京都大田区西6郷4-30-7音金6郷マンション1-C号,東京都,大田区
...,...,...,...
57,名古屋市熱田区神宮4-7-27号,,名古屋市
58,名古屋市熱田区神宮4-7-27号Bビル,,名古屋市
59,名古屋市港区空見町36,,名古屋市
60,名古屋市港区101屋3-63,,名古屋市


In [7]:
non_city_address_data
# ここからさらに小さな行政単位を取り除く

towns: list = []
non_town_address_data: list = []

for town_address_data in non_city_address_data:
    if "区" in town_address_data:
        index: int = town_address_data.find("区")
        towns.append(town_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_town_address_data.append(town_address_data[index+1:])
    elif "郡" in town_address_data:
        index: int = town_address_data.find("郡")
        towns.append(town_address_data[0:index+1])
        if len(town_address_data) > index+1:
            # index out of rangeを避けるため
            non_town_address_data.append(town_address_data[index+1:])
    elif "町" in town_address_data:
        index: int = town_address_data.find("町")
        towns.append(town_address_data[0:index+1])
        if len(town_address_data) > index+1:
            # index out of rangeを避けるため
            non_town_address_data.append(town_address_data[index+1:])
    elif "村" in town_address_data:
        index: int = town_address_data.find("村")
        towns.append(town_address_data[0:index+1])
        if len(town_address_data) > index+1:
            # index out of rangeを避けるため
            non_town_address_data.append(town_address_data[index+1:])
    else:
        towns.append("")
        non_town_address_data.append(town_address_data)

data['town'] = towns
# data['other'] = non_town_address_data


pd.DataFrame(data)
# city town districtに数字か-があったら要注意

,original,prefacture,city,town
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,
3,東京都品川区高輪4-8-10-5階,東京都,品川区,
4,東京都大田区西6郷4-30-7音金6郷マンション1-C号,東京都,大田区,
...,...,...,...,...
57,名古屋市熱田区神宮4-7-27号,,名古屋市,熱田区
58,名古屋市熱田区神宮4-7-27号Bビル,,名古屋市,熱田区
59,名古屋市港区空見町36,,名古屋市,港区
60,名古屋市港区101屋3-63,,名古屋市,港区


In [8]:
non_town_address_data

districts: list = []
others: list = []

def extract_district(string: str):
    district_numbers:list = []
    for i in range(0,10):
        district_numbers.append(str(i))
    # 数字が現れるまでを記録する
    for i in range(len(string)):
        if string[i] in district_numbers:
            return (string[0:i],string[i:])
    return ("",string)
        
for district_data in non_town_address_data:
    district_tuple_data = extract_district(district_data)
    districts.append(district_tuple_data[0])
    others.append(district_tuple_data[1])

data['district'] = districts
# data['others'] = others
pd.set_option("display.max_rows",100)
pd.DataFrame(data)

# 全角の2が混ざっている

,original,prefacture,city,town,district
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町,
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,,山王
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,,浅草橋
3,東京都品川区高輪4-8-10-5階,東京都,品川区,,高輪
4,東京都大田区西6郷4-30-7音金6郷マンション1-C号,東京都,大田区,,西
5,北海道蛭田郡豊浦町字桜207,北海道,蛭田郡,豊浦町,字桜
6,東京都千代田区1-町8,東京都,千代田区,1-町,
7,豊平区豊平4条3-3-10-403,,豊平区,,豊平
8,住吉1-17-18アルぺ-ジュ401,,,,住吉
9,本町田2428-ハ-10-109,,本町,,田


In [9]:
others
# ここから1-3-5などを抜き取る
house_numbers: list = []
others_head: list = []
others_tail: list = []

def extract_house_number(string: str):
    index = re.search('(([0-9]+)-)+([0-9]+)',string)
    if index == None:
        house_numbers.append("")
        others_head.append("")
        others_tail.append(string)
    else:
        house_numbers.append(string[index.start():index.end()])
        others_head.append(string[0:index.start()])
        if index.end()+1 < len(string): others_tail.append(string[index.end():])
        else: others_tail.append("")

for other in others:
    extract_house_number(other)

data["others_head"] =  others_head
data["house_number"] = house_numbers
# data["others_tail"] = others_tail

pd.DataFrame(data)

,original,prefacture,city,town,district,others_head,house_number
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町,,,1-8-12
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,,山王,,1-7-8
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,,浅草橋,,1-32-6-2
3,東京都品川区高輪4-8-10-5階,東京都,品川区,,高輪,,4-8-10-5
4,東京都大田区西6郷4-30-7音金6郷マンション1-C号,東京都,大田区,,西,6郷,4-30-7
5,北海道蛭田郡豊浦町字桜207,北海道,蛭田郡,豊浦町,字桜,,
6,東京都千代田区1-町8,東京都,千代田区,1-町,,,
7,豊平区豊平4条3-3-10-403,,豊平区,,豊平,4条,3-3-10-403
8,住吉1-17-18アルぺ-ジュ401,,,,住吉,,1-17-18
9,本町田2428-ハ-10-109,,本町,,田,2428-ハ-,10-109


In [10]:
# 数字が other_tailにあるのでそれを補完
# 名前のところが - になっている, でもそうでないとうまく分類分けできない

others_tail
munipulated_others_tail: list = []

for id in range(len(others_tail)):
    if re.match('[0-9]+$',others_tail[id]):
        print(others_tail[id])
        data['house_number'][id] = data['house_number'][id] + others_tail[id]
        munipulated_others_tail.append("")
    else:
        munipulated_others_tail.append(others_tail[id])

data['others_tail'] = munipulated_others_tail
pd.DataFrame(data)



207
8
1
86
36


,original,prefacture,city,town,district,others_head,house_number,others_tail
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町,,,1-8-12,-S-1203
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,,山王,,1-7-8,山王薬局1丁目店
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,,浅草橋,,1-32-6-2,
3,東京都品川区高輪4-8-10-5階,東京都,品川区,,高輪,,4-8-10-5,
4,東京都大田区西6郷4-30-7音金6郷マンション1-C号,東京都,大田区,,西,6郷,4-30-7,音金6郷マンション1-C号
5,北海道蛭田郡豊浦町字桜207,北海道,蛭田郡,豊浦町,字桜,,207,
6,東京都千代田区1-町8,東京都,千代田区,1-町,,,8,
7,豊平区豊平4条3-3-10-403,,豊平区,,豊平,4条,3-3-10-403,
8,住吉1-17-18アルぺ-ジュ401,,,,住吉,,1-17-18,アルぺ-ジュ401
9,本町田2428-ハ-10-109,,本町,,田,2428-ハ-,10-109,


In [13]:
def extract_number_and_alphabet1(string: str):
    if(string==""): return None
    return re.match('^(-[A-Z 0-9]+)*([A-Z 0-9]+)*$', string)

print(extract_number_and_alphabet1("-R2-104"))
print(extract_number_and_alphabet1('540-C-402'))
print(extract_number_and_alphabet1('-C'))
print(extract_number_and_alphabet1('20B-12'))
print(extract_number_and_alphabet1('-Ｃ1-502')) # Cが全角?
print(extract_number_and_alphabet1('-S-1203'))

print('##################')
def extract_number_and_alphabet2(string: str):
    if(string==""): return None
    return re.match('^([A-Z 0-9]+)*(-[A-Z 0-9]+)*$',string)

print(extract_number_and_alphabet2('540-C-402'))
print(extract_number_and_alphabet2('-Ｃ1-502'))
print(extract_number_and_alphabet2('-S-1203'))
print(extract_number_and_alphabet2('20B-12'))

print('########')

print('Ｃ'=='C')
# 全角

<re.Match object; span=(0, 7), match='-R2-104'>
None
<re.Match object; span=(0, 2), match='-C'>
None
None
<re.Match object; span=(0, 7), match='-S-1203'>
##################
<re.Match object; span=(0, 9), match='540-C-402'>
None
<re.Match object; span=(0, 7), match='-S-1203'>
<re.Match object; span=(0, 6), match='20B-12'>
########
False


In [11]:
df = pd.DataFrame(data)
df.to_csv('output/output.csv',encoding='utf-8_sig')

# 文字化けについて https://qiita.com/y4m3/items/674423b596284bbc7cf7